<a href="https://colab.research.google.com/github/cbrss/Programacion_concurrente_M1/blob/main/PC_TP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codigo en: **C**

In [58]:
%%writefile fork_tree.c

#include <stdlib.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/wait.h>

#include <string.h>
#include <linux/prctl.h>
#include <sys/prctl.h>

#define EXIT_SUCCESS 0
#define TIMETOSLEEP 30

void errorChild(char* nameChild);
void changeName(char* name);
void sleepAndWait(int cantWaits);

int main()
{
    int i;
    pid_t pid = fork();


    if(pid<0)
    {
        errorChild("B");
        return EXIT_FAILURE;
    }
    if(pid)//A
    {
        changeName("A");
        sleepAndWait(1);
        return EXIT_SUCCESS;
    }
    //B
    changeName("B");
    pid=fork();
    if(pid<0)
    {
        errorChild("C");
        return EXIT_FAILURE;
    }
    if(!pid) //C
    {
        changeName("C");
        pid=fork();
        if(pid<0)
        {
            errorChild("E");
            return EXIT_FAILURE;
        }
        if(pid) //C
        {
            sleepAndWait(1);
            return EXIT_SUCCESS;
        }
        //E
        changeName("E");
        for(i=0; i<2; i++) //H - I
        {
            pid=fork();
            if(pid<0)
            {
                errorChild(i==0?"H":"I");
                return EXIT_FAILURE;
            }
            if(!pid)
            {
                changeName(i==0?"H":"I");
                sleepAndWait(0);
                return EXIT_SUCCESS;
            }
        }
        sleepAndWait(2);
        return EXIT_SUCCESS;
    }

    //sigue B PARA D
    pid=fork();
    if(pid<0)
    {
        errorChild("D");
        return EXIT_FAILURE;
    }
    if(!pid) // D
    {
        changeName("D");
        for(i=0; i<2; i++) //F - G
        {
            pid=fork();
            if(pid<0)
            {
                errorChild(i==0?"F":"G");
                return EXIT_FAILURE;
            }
            if(!pid)
            {
                changeName(i==0?"F":"G");
                sleepAndWait(0);
                return EXIT_SUCCESS;
            }
        }
        sleepAndWait(2);
        return EXIT_SUCCESS;
    }
    //sigue B
    sleepAndWait(2);
    return EXIT_SUCCESS;
}

void errorChild(char* nameChild)
{
  printf("Error de creación de %s\n",nameChild);
}

void changeName(char* name)
{
  char auxCad[16]="NameDefault";
  prctl(PR_SET_NAME, strcpy(auxCad,name),NULL,NULL,NULL);
}

void sleepAndWait(int cantWaits)
{
  int i;
  sleep(TIMETOSLEEP);
  for(i=0;i<cantWaits;i++)
    wait(NULL);
}

Writing fork_tree.c


Procederemos a compilar el programa:

In [59]:
!gcc -o treeC fork_tree.c

Y ahora a ejecutarlo:

In [60]:
!nohup ./treeC 1>/dev/null 2>/dev/null &

Lo hacemos en 2do plano y con la redireccion de la salida para poder ejecutar el siguiente comando y visualizar el arbol de procesos generado:

In [62]:
!pstree -pc $(pgrep -f "treeC" | head -1)

A(2182)───B(2183)─┬─C(2184)───E(2188)─┬─H(2189)
                  │                   └─I(2190)
                  └─D(2185)─┬─F(2186)
                            └─G(2187)


# Codigo en: **Java**

In [43]:
%%writefile MainProcess.java
package processes;

import java.io.IOException;
import java.util.ArrayList;
import java.util.List;

// para ver los procesos en ps: Get-Process | Where-Object { $_.Name -like "java*" }


public class MainProcess {

	public static final int TIME_VERIFICATION = 4000;

	public static void main(String[] args) throws IOException, InterruptedException {

		ProcessHandle process = ProcessHandle.current();
		long pid = process.pid();
    long ppid = process.parent().get().pid();
    String processName="";
    if(args.length == 0) {
      processName = "A";
    } else {
      processName = args[0];
    }
    System.out.println("Proceso: "+ processName + " PID:" + pid + " PID Padre: " + ppid);

    List<Process> childs = new ArrayList<>();
    if(processName.equals("A")) {
      childs.add(createProcess("B"));
    }
    else if(processName.equals("B")){
      childs.add(createProcess("C"));
      childs.add(createProcess("D"));
    }
    else if(processName.equals("C")) {
      childs.add(createProcess("E"));
    }
    else if(processName.equals("E")) {
      childs.add(createProcess("H"));
      childs.add(createProcess("I"));
    }
    else if(processName.equals("D")) {
      childs.add(createProcess("F"));
      childs.add(createProcess("G"));
    }

    Thread.sleep(TIME_VERIFICATION);	//duerme todos los procesos
    for(Process child : childs) {
      child.waitFor(); // el proceso solo continua si el proceso hijo termina
    }
	}

	private static Process createProcess(String processName) throws IOException, InterruptedException {
		ProcessBuilder pb = new ProcessBuilder("java", "MainProcess.java", processName);
		pb.inheritIO();
		Process process = pb.start();
		return process;
	}
}


Writing MainProcess.java


Procedemos a ejecutar el programa:

In [44]:
!nohup java MainProcess.java >out.txt &

nohup: redirecting stderr to stdout


Como mencionamos antes, se ejecutara en 2do plano para poder visualizar el arbol de procesos:

In [55]:
!ps -ef | grep MainProcess.java

root        1826       1 17 13:07 ?        00:00:03 java MainProcess.java
root        1856    1826 15 13:07 ?        00:00:02 java MainProcess.java B
root        1890    1856 19 13:07 ?        00:00:02 java MainProcess.java C
root        1949    1890 31 13:07 ?        00:00:03 java MainProcess.java E
root        2033    1949 50 13:07 ?        00:00:03 java MainProcess.java H
root        2036    1949 42 13:07 ?        00:00:02 java MainProcess.java I
root        2102     327  0 13:07 ?        00:00:00 /bin/bash -c ps -ef | grep MainProcess.java
root        2104    2102  0 13:07 ?        00:00:00 grep MainProcess.java


Ademas, el siguiente comando muestra la salida del programa:

In [56]:
!cat out.txt

Proceso: A PID:1826 PID Padre: 1
Proceso: B PID:1856 PID Padre: 1826
Proceso: C PID:1890 PID Padre: 1856
Proceso: D PID:1893 PID Padre: 1856
Proceso: E PID:1949 PID Padre: 1890
Proceso: G PID:1970 PID Padre: 1893
Proceso: F PID:1960 PID Padre: 1893
Proceso: H PID:2033 PID Padre: 1949
Proceso: I PID:2036 PID Padre: 1949


# Codigo en: **Python**

In [ ]:
%%writefile tree.py

import time
import os

PADDING = 4
SLEEP_TIME = 5


def print_proc(name):
    st = f"{name} {os.getpid()}"
    extra_padding = 0

    if name == "B":
        extra_padding = 1
    elif name in ("C", "D"):
        extra_padding = 2
    elif name in ("E", "F", "G"):
        extra_padding = 3
    elif name in ("H", "I"):
        extra_padding = 4

    print(st.rjust(extra_padding*PADDING+len(st)))


def create_leaf(name):
    print_proc(name)
    time.sleep(SLEEP_TIME)


if __name__ == "__main__":
    try:
        print_proc("A")

        new_proc = os.fork()
        if new_proc != 0:
            # Seguimos en el padre (A)
            os.wait()  # Que espere a los hijos
        else:
            # Estamos en el hijo (B)
            print_proc("B")

            new_proc = os.fork()
            if new_proc == 0:
                # Estamos en el 1er hijo (C)
                print_proc("C")

                new_proc = os.fork()
                if new_proc != 0:
                    # Seguimos en el padre (C)
                    os.wait()
                else:
                    # Estamos en el hijo (E)
                    print_proc("E")

                    new_proc = os.fork()
                    if new_proc == 0:
                        create_leaf("H")
                    else:
                        new_proc = os.fork()

                        if new_proc != 0:
                            os.wait()
                        else:
                            create_leaf("I")
            else:
                # Seguimos en el padre (B)
                new_proc = os.fork()
                if new_proc != 0:
                    # Seguimos en el padre (B)
                    os.wait()
                else:
                    # Estamos en el 2do hijo (D)
                    print_proc("D")

                    new_proc = os.fork()
                    if new_proc == 0:
                        create_leaf("F")
                    else:
                        new_proc = os.fork()
                        if new_proc != 0:
                            os.wait()
                        else:
                            create_leaf("G")

    except Exception as e:
        print(e)


Ahora lo ejecutamos en 2do plano:

In [34]:
!nohup python3 tree.py > tree.txt &

nohup: redirecting stderr to stdout


Con el siguiente comando vemos el arbol generado

In [35]:
!ps -ef | grep tree.py

root        1365       1  1 13:05 ?        00:00:00 python3 tree.py
root        1366    1365  0 13:05 ?        00:00:00 python3 tree.py
root        1367    1366  0 13:05 ?        00:00:00 python3 tree.py
root        1368    1366  0 13:05 ?        00:00:00 python3 tree.py
root        1369    1367  0 13:05 ?        00:00:00 python3 tree.py
root        1370    1368  0 13:05 ?        00:00:00 python3 tree.py
root        1371    1369  0 13:05 ?        00:00:00 python3 tree.py
root        1372    1368  0 13:05 ?        00:00:00 python3 tree.py
root        1373    1369  0 13:05 ?        00:00:00 python3 tree.py
root        1378     327  0 13:05 ?        00:00:00 /bin/bash -c ps -ef | grep tree.py
root        1380    1378  0 13:05 ?        00:00:00 grep tree.py


Y la salida del archivo:

In [ ]:
!cat tree.txt